In [50]:
#Data exploration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error

In [51]:
df = pd.read_csv("./steam.csv")
df.describe()

,appid,english,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,price
count,2.707500e+04,27075.000000,27075.000000,27075.000000,2.707500e+04,27075.000000,27075.000000,27075.00000,27075.000000
mean,5.962035e+05,0.981127,0.354903,45.248864,1.000559e+03,211.027147,149.804949,146.05603,6.078193
std,2.508942e+05,0.136081,2.406044,352.670281,1.898872e+04,4284.938531,1827.038141,2353.88008,7.874922
min,1.000000e+01,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.00000,0.000000
25%,4.012300e+05,1.000000,0.000000,0.000000,6.000000e+00,2.000000,0.000000,0.00000,1.690000
50%,5.990700e+05,1.000000,0.000000,7.000000,2.400000e+01,9.000000,0.000000,0.00000,3.990000
75%,7.987600e+05,1.000000,0.000000,23.000000,1.260000e+02,42.000000,0.000000,0.00000,7.190000
max,1.069460e+06,1.000000,18.000000,9821.000000,2.644404e+06,487076.000000,190625.000000,190625.00000,421.990000


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27075 entries, 0 to 27074
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   appid             27075 non-null  int64  
 1   name              27075 non-null  object 
 2   release_date      27075 non-null  object 
 3   english           27075 non-null  int64  
 4   developer         27075 non-null  object 
 5   publisher         27075 non-null  object 
 6   platforms         27075 non-null  object 
 7   required_age      27075 non-null  int64  
 8   categories        27075 non-null  object 
 9   genres            27075 non-null  object 
 10  steamspy_tags     27075 non-null  object 
 11  achievements      27075 non-null  int64  
 12  positive_ratings  27075 non-null  int64  
 13  negative_ratings  27075 non-null  int64  
 14  average_playtime  27075 non-null  int64  
 15  median_playtime   27075 non-null  int64  
 16  owners            27075 non-null  object

In [53]:
# preprocessing(1). drop the columns less than 1500 positive ratings
df = df[df['positive_ratings'] > 15000]
df = df.reset_index(drop = True)
df.tail(20)

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
255,582160,Assassin's Creed® Origins,2017-10-26,1,Ubisoft Montreal,Ubisoft,windows,0,Single-player;Steam Achievements;Steam Trading...,Action;Adventure;RPG,Open World;Assassin;Action,67,31644,6007,2908,3143,1000000-2000000,49.99
256,588430,Fallout Shelter,2017-03-29,1,Bethesda Game Studios,Bethesda Softworks,windows,0,Single-player;Steam Achievements;Full controll...,Free to Play;RPG;Simulation,Free to Play;Survival;Base-Building,35,21481,3569,1116,93,2000000-5000000,0.00
257,588650,Dead Cells,2018-08-06,1,Motion Twin,Motion Twin,windows;mac;linux,0,Single-player;Steam Achievements;Full controll...,Action;Indie,Rogue-like;Pixel Graphics;Metroidvania,60,26033,1434,1006,973,1000000-2000000,21.99
258,594570,Total War: WARHAMMER II,2017-09-28,1,CREATIVE ASSEMBLY;Feral Interactive (Mac);Fera...,SEGA;Feral Interactive (Mac);Feral Interactive...,windows;mac;linux,0,Single-player;Multi-player;Steam Achievements;...,Action;Strategy,Strategy;Fantasy;RTS,134,19420,2994,4721,2624,1000000-2000000,39.99
259,594650,Hunt Showdown,2018-02-22,1,Crytek,Crytek,windows,0,Online Multi-Player;Online Co-op,Action;Early Access,Early Access;Multiplayer;Survival,0,20679,7074,991,1169,1000000-2000000,25.99
260,601150,Devil May Cry 5,2019-03-07,1,"CAPCOM Co., Ltd.","CAPCOM Co., Ltd.",windows,18,Single-player;Online Multi-Player;Online Co-op...,Action,Action;Hack and Slash;Character Action Game,51,17657,1487,916,950,1000000-2000000,44.99
261,611500,Quake Champions,2017-08-22,1,id Software,Bethesda Softworks,windows,0,Online Multi-Player;Steam Achievements,Action;Free to Play;Early Access,Early Access;FPS;Arena Shooter,52,15879,7098,754,102,500000-1000000,0.00
262,632360,Risk of Rain 2,2019-03-28,1,Hopoo Games,Gearbox Publishing,windows,0,Single-player;Online Multi-Player;Online Co-op...,Action;Indie;Early Access,Early Access;Rogue-like;Multiplayer,0,17902,975,1455,1323,1000000-2000000,14.99
263,644560,Mirror,2018-04-19,1,KAGAMI WORKs,Paradise Project,windows;mac,0,Single-player;Steam Achievements;Steam Trading...,Adventure;Indie;RPG,Sexual Content;Nudity;Mature,114,21474,702,281,291,500000-1000000,1.69
264,646570,Slay the Spire,2019-01-23,1,Mega Crit Games,Mega Crit Games,windows;mac;linux,0,Single-player;Steam Achievements;Full controll...,Indie;Strategy,Card Game;Turn-Based;Rogue-like,43,26166,998,3034,1857,1000000-2000000,19.49


In [54]:
# preprocessing(2). Drop non-English(0) columns that bring error in name 
df = df[df['english'] == 1]
df = df.reset_index(drop = True)
df.tail(20)

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
254,582010,MONSTER HUNTER: WORLD,2018-08-09,1,"CAPCOM Co., Ltd.","CAPCOM Co., Ltd.",windows,0,Single-player;Multi-player;Co-op;Steam Achieve...,Action,Action;Hunting;Multiplayer,50,72029,35294,6512,5372,5000000-10000000,49.99
255,582160,Assassin's Creed® Origins,2017-10-26,1,Ubisoft Montreal,Ubisoft,windows,0,Single-player;Steam Achievements;Steam Trading...,Action;Adventure;RPG,Open World;Assassin;Action,67,31644,6007,2908,3143,1000000-2000000,49.99
256,588430,Fallout Shelter,2017-03-29,1,Bethesda Game Studios,Bethesda Softworks,windows,0,Single-player;Steam Achievements;Full controll...,Free to Play;RPG;Simulation,Free to Play;Survival;Base-Building,35,21481,3569,1116,93,2000000-5000000,0.00
257,588650,Dead Cells,2018-08-06,1,Motion Twin,Motion Twin,windows;mac;linux,0,Single-player;Steam Achievements;Full controll...,Action;Indie,Rogue-like;Pixel Graphics;Metroidvania,60,26033,1434,1006,973,1000000-2000000,21.99
258,594570,Total War: WARHAMMER II,2017-09-28,1,CREATIVE ASSEMBLY;Feral Interactive (Mac);Fera...,SEGA;Feral Interactive (Mac);Feral Interactive...,windows;mac;linux,0,Single-player;Multi-player;Steam Achievements;...,Action;Strategy,Strategy;Fantasy;RTS,134,19420,2994,4721,2624,1000000-2000000,39.99
259,594650,Hunt Showdown,2018-02-22,1,Crytek,Crytek,windows,0,Online Multi-Player;Online Co-op,Action;Early Access,Early Access;Multiplayer;Survival,0,20679,7074,991,1169,1000000-2000000,25.99
260,601150,Devil May Cry 5,2019-03-07,1,"CAPCOM Co., Ltd.","CAPCOM Co., Ltd.",windows,18,Single-player;Online Multi-Player;Online Co-op...,Action,Action;Hack and Slash;Character Action Game,51,17657,1487,916,950,1000000-2000000,44.99
261,611500,Quake Champions,2017-08-22,1,id Software,Bethesda Softworks,windows,0,Online Multi-Player;Steam Achievements,Action;Free to Play;Early Access,Early Access;FPS;Arena Shooter,52,15879,7098,754,102,500000-1000000,0.00
262,632360,Risk of Rain 2,2019-03-28,1,Hopoo Games,Gearbox Publishing,windows,0,Single-player;Online Multi-Player;Online Co-op...,Action;Indie;Early Access,Early Access;Rogue-like;Multiplayer,0,17902,975,1455,1323,1000000-2000000,14.99
263,644560,Mirror,2018-04-19,1,KAGAMI WORKs,Paradise Project,windows;mac,0,Single-player;Steam Achievements;Steam Trading...,Adventure;Indie;RPG,Sexual Content;Nudity;Mature,114,21474,702,281,291,500000-1000000,1.69


In [55]:
# preprocessing(3). make "relrease_year"row and drop release_date
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_date'].dt.year.value_counts()

2016    38
2015    38
2017    36
2014    33
2013    31
2012    26
2018    23
2011    14
2010     9
2019     7
2009     6
2008     6
2007     2
2004     2
2000     1
2006     1
1998     1
Name: release_date, dtype: int64

In [56]:
df['release_year'] = df['release_date'].dt.year
df = df.drop(['release_date'], axis=1)
df.head()

,appid,name,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,release_year
0,10,Counter-Strike,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19,2000
1,70,Half-Life,1,Valve,Valve,windows;mac;linux,0,Single-player;Multi-player;Online Multi-Player...,Action,FPS;Classic;Action,0,27755,1100,1300,83,5000000-10000000,7.19,1998
2,220,Half-Life 2,1,Valve,Valve,windows;mac;linux,0,Single-player;Steam Achievements;Steam Trading...,Action,FPS;Action;Sci-fi,33,67902,2419,691,402,10000000-20000000,7.19,2004
3,240,Counter-Strike: Source,1,Valve,Valve,windows;mac;linux,0,Multi-player;Cross-Platform Multiplayer;Steam ...,Action,Action;FPS;Multiplayer,147,76640,3497,6842,400,10000000-20000000,7.19,2004
4,400,Portal,1,Valve,Valve,windows;mac;linux,0,Single-player;Steam Achievements;Captions avai...,Action,Puzzle;First-Person;Singleplayer,15,51801,1080,288,137,10000000-20000000,7.19,2007


In [124]:
#categories and genres rows seems important for content similiarity.
#combine the categories and genres in one column
df['cateNgenre'] = df['categories'] + ';' + df['genres']

#vectorize with tfidfVectorizer(1)
tf = TfidfVectorizer(stop_words='english')
dtm = tf.fit_transform(df['cateNgenre'].values.astype('U'))
dtm = pd.DataFrame(dtm.todense(), columns=tf.get_feature_names())
dtm.head()

,access,achievements,action,adventure,animation,anti,app,available,captions,cards,...,steamvr,strategy,support,trading,utilities,valve,video,violent,vr,workshop
0,0.0,0.000000,0.136509,0.0,0.0,0.324562,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.324562,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.156371,0.0,0.0,0.371786,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.371786,0.0,0.0,0.0,0.0
2,0.0,0.124072,0.130324,0.0,0.0,0.000000,0.0,0.334774,0.334774,0.138615,...,0.0,0.0,0.136475,0.138615,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.108671,0.114147,0.0,0.0,0.271395,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.271395,0.0,0.0,0.0,0.0
4,0.0,0.095086,0.099878,0.0,0.0,0.000000,0.0,0.513128,0.256564,0.000000,...,0.0,0.0,0.104592,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [125]:
#vectorize with CountVectorizer
cv = CountVectorizer(ngram_range=(1,3))
dcm = cv.fit_transform(df['cateNgenre'])
dcm = pd.DataFrame(dcm.todense(), columns=cv.get_feature_names())
dcm.head()

,access,achievements,achievements action,achievements action free,achievements captions,achievements captions available,achievements full,achievements full controller,achievements in,achievements in app,...,workshop stats includes,workshop stats steam,workshop steam,workshop steam cloud,workshop steam leaderboards,workshop steamvr,workshop steamvr collectibles,workshop strategy,workshop valve,workshop valve anti
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [136]:
important_columns = ['categories', 'genres', 'name']
df['important_feature_'] = df['categories']+";"+df["genres"]
df["appid"] = range(0, df.shape[0])
cm = CountVectorizer().fit_transform(df['important_feature'])
cs = cosine_similarity(cm)
cs.shape

(274, 274)

In [135]:
input_name = 'Mirror'
app_id = df[df.name == input_name]['appid'].values[0]
print(app_id)

263


In [142]:
scores = list(enumerate(cs[app_id]))
scores = sorted(scores,key = lambda x:x[1],reverse = True)
scores = scores[1:]
j = 0
print ('10 recomended game for',input_name,'are:\n')
for item in scores:
    game_title = (df[df.appid == item[0]]['name'].values[0])
    print(j+1,game_title)
    print('Similarity : ',end='')
    print(item[1])
    j = j+1
    if j>9 :
        break

10 recomended game for Mirror are:

1 To the Moon
Similarity : 0.9258200997725516
2 Papers, Please
Similarity : 0.9245003270420487
3 The Binding of Isaac
Similarity : 0.9152086306448589
4 Dust: An Elysian Tail
Similarity : 0.904534033733291
5 Firewatch
Similarity : 0.895978670381041
6 Bastion
Similarity : 0.8753762190648171
7 This War of Mine
Similarity : 0.8753762190648171
8 SOMA
Similarity : 0.8753762190648171
9 Transistor
Similarity : 0.8743856497851876
10 Darkest Dungeon®
Similarity : 0.8743856497851876


In [143]:
tm = TfidfVectorizer(stop_words='english').fit_transform(df['important_feature'])
tcs = cosine_similarity(tm)
score_with_tm = list(enumerate(tcs[app_id]))
score_with_tm = sorted(score_with_tm,key = lambda x:x[1],reverse = True)
score_with_tm = scores[1:]

In [144]:
j = 0
print ('10 recomended game for',input_name,'are:\n')
for item in score_with_tm:
    game_title = (df[df.appid == item[0]]['name'].values[0])
    print(j+1,game_title)
    print('Similarity : ',end='')
    print(item[1])
    j = j+1
    if j>9 :
        break

10 recomended game for Mirror are:

1 Papers, Please
Similarity : 0.9245003270420487
2 The Binding of Isaac
Similarity : 0.9152086306448589
3 Dust: An Elysian Tail
Similarity : 0.904534033733291
4 Firewatch
Similarity : 0.895978670381041
5 Bastion
Similarity : 0.8753762190648171
6 This War of Mine
Similarity : 0.8753762190648171
7 SOMA
Similarity : 0.8753762190648171
8 Transistor
Similarity : 0.8743856497851876
9 Darkest Dungeon®
Similarity : 0.8743856497851876
10 A Story About My Uncle
Similarity : 0.8743856497851876


In [57]:
#make important_feature row that make the decision of similiarity with contents
df['important_feature'] = df['categories']+";"+df["genres"]
#use appid as index
df["appid"] = range(0, df.shape[0])

#vectorize with CountVectorizer
cm = CountVectorizer().fit_transform(df['important_feature'])
#vectorize with CountVectorizer
tm = TfidfVectorizer(stop_words='english').fit_transform(df['important_feature'])

input_name = 'Mirror'
def recommendation_cosine(vectorize, input_name, df):
    cs = cosine_similarity(vectorize)
    app_id = df[df.name == input_name]['appid'].values[0]
    score_list = list(enumerate(cs[app_id]))
    score_list = sorted(score_list,key = lambda x:x[1],reverse = True)
    score_list = score_list[1:]
    j = 0
    print("calculating similiarity using cosine similarity")
    print ('10 recomended game for',input_name,'are:\n')
    for item in score_list:
        game_title = (df[df.appid == item[0]]['name'].values[0])
        print(j+1,game_title)
        print('Similarity : ',end='')
        print(item[1])
        j = j+1
        if j>9 :
            break
    print("\n\n")
recommendation_cosine(cm, input_name, df)
recommendation_cosine(tm, input_name, df)

calculating similiarity using cosine similarity
10 recomended game for Mirror are:

1 To the Moon
Similarity : 0.9258200997725516
2 Papers, Please
Similarity : 0.9245003270420487
3 The Binding of Isaac
Similarity : 0.9152086306448589
4 Dust: An Elysian Tail
Similarity : 0.904534033733291
5 Firewatch
Similarity : 0.895978670381041
6 Bastion
Similarity : 0.8753762190648171
7 This War of Mine
Similarity : 0.8753762190648171
8 SOMA
Similarity : 0.8753762190648171
9 Transistor
Similarity : 0.8743856497851876
10 Darkest Dungeon®
Similarity : 0.8743856497851876



calculating similiarity using cosine similarity
10 recomended game for Mirror are:

1 To the Moon
Similarity : 0.9490635279386663
2 Dust: An Elysian Tail
Similarity : 0.9283642873947403
3 The Binding of Isaac
Similarity : 0.9155938951602519
4 Kingdom Come: Deliverance
Similarity : 0.8736709053725482
5 Transistor
Similarity : 0.8731375870707236
6 Grim Dawn
Similarity : 0.8585910749884802
7 Firewatch
Similarity : 0.8541621377123392
8 

In [58]:
def recommend_euclidian(vectorize, input_name, df, n=10):
    app_id = df[df.name == input_name]['appid'].values[0]
    #df['similarity'] = euclidean_distances([np.array(df.iloc[row_number,:-1])],Y=df.iloc[:,:-1]).reshape(-1,1)  

    es =euclidean_distances(vectorize, vectorize)

    score_list = list(enumerate(es[app_id]))
    score_list = sorted(score_list,key = lambda x:x[1],reverse = True)
    score_list = score_list[1:]
    j = 0
    print("calculating similarity using euclidean_distances")
    print ('10 recomended game for',input_name,'are:\n')
    for item in score_list:
        game_title = (df[df.appid == item[0]]['name'].values[0])
        print(j+1,game_title)
        print('Similarity : ',end='')
        print(item[1])
        j = j+1
        if j>9 :
            break
    print('\n\n')
    
recommend_euclidian(cm, input_name, df, 1)
recommend_euclidian(tm, input_name, df, 1)

calculating similarity using euclidean_distances
10 recomended game for Mirror are:

1 BattleBlock Theater®
Similarity : 7.937253933193772
2 Arma 3
Similarity : 7.874007874011811
3 Brawlhalla
Similarity : 7.810249675906654
4 Divinity: Original Sin 2 - Definitive Edition
Similarity : 7.681145747868608
5 Castle Crashers®
Similarity : 7.54983443527075
6 VRChat
Similarity : 7.280109889280518
7 No More Room in Hell
Similarity : 6.4031242374328485
8 Killing Floor 2
Similarity : 6.244997998398398
9 Fistful of Frags
Similarity : 6.244997998398398
10 Stick Fight: The Game
Similarity : 6.244997998398398



calculating similarity using euclidean_distances
10 recomended game for Mirror are:

1 Hunt Showdown
Similarity : 1.3983951232122245
2 Realm Royale
Similarity : 1.385743647314257
3 Counter-Strike
Similarity : 1.3767647874111815
4 Source Filmmaker
Similarity : 1.3749540539344238
5 Arma 2: Operation Arrowhead
Similarity : 1.3689413694219126
6 Heroes & Generals
Similarity : 1.3494884062136154
7 H

In [60]:
# preprocessing(4). split genres with ; and make rows with unique genres. Then drop steamspy_tags that is similiar with genre

i=0
df_k = pd.DataFrame(df['appid'])
for genre in df['important_feature']:
    genre_list = genre.split(";")
    for gen in genre_list:
        if(gen in df_k.columns):
            df_k[gen].iloc[i]=1
        else:
            df_k[gen] =0
            df_k[gen].iloc[i]=1    
    i+=1
df_k

C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,appid,Multi-player,Online Multi-Player,Local Multi-Player,Valve Anti-Cheat enabled,Action,Single-player,Steam Cloud,Steam Achievements,Steam Trading Cards,...,Early Access,Racing,Sports,VR Support,Nudity,Violent,Gore,Sexual Content,Design & Illustration,Utilities
0,0,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,3,1,0,0,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,269,0,0,0,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
270,270,1,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
271,271,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
272,272,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
df_k = df_k.drop(['appid'], axis=1)
df_k

,Multi-player,Online Multi-Player,Local Multi-Player,Valve Anti-Cheat enabled,Action,Single-player,Steam Cloud,Steam Achievements,Steam Trading Cards,Captions available,...,Early Access,Racing,Sports,VR Support,Nudity,Violent,Gore,Sexual Content,Design & Illustration,Utilities
0,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,0,0,0,0,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
270,1,1,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
271,0,0,0,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
272,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
def recommdend_knn(input_name, df,df_k, n=11):
    app_id = df[df.name == input_name]['appid'].values[0]
    model = NearestNeighbors(n_neighbors= (n+1))
    model.fit(df_k)
    distances, indices = model.kneighbors([df_k.iloc[app_id]])
    recommend_knn = sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key = lambda x: x[1],reverse = True)[:0:-1]
    
    print("calculating top ten recommended game using knn")
    print ('10 recommended game for',input_name,'are:\n')
    j=0
    for item in recommend_knn:
        if(j==0):#j==0 is distnace 0. It means a target row.
            j = j+1
        else:
            game_title = (df[df.appid == item[0]]['name'].values[0])
            print(j,game_title)
            print('Similarity : ',end='')
            print(item[1])
            j = j+1
        if j>10:
            break
    print('\n\n')
recommdend_knn("Mirror", df, df_k)

calculating top ten recommended game using knn
10 recommended game for Mirror are:

1 To the Moon
Similarity : 1.0
2 The Binding of Isaac
Similarity : 1.4142135623730951
3 Firewatch
Similarity : 1.4142135623730951
4 Dust: An Elysian Tail
Similarity : 1.4142135623730951
5 Hellblade: Senua's Sacrifice
Similarity : 1.7320508075688772
6 Papers, Please
Similarity : 1.7320508075688772
7 Undertale
Similarity : 1.7320508075688772
8 Kingdom Come: Deliverance
Similarity : 1.7320508075688772
9 Transistor
Similarity : 1.7320508075688772
10 Hollow Knight
Similarity : 1.7320508075688772



